###  شرح پروژه:
در این پروژه، یک گراف وزن‌دار به همراه تابع هیوریستیک در اختیار شما قرار گرفته است. شما باید مسیر بهینه از یک گره مبدأ (که کاربر وارد می‌کند) تا گره مقصد مشخص‌شده (`ControlBase`) را با استفاده از دو الگوریتم زیر بیابید:

- الگوریتم **IDA***
- الگوریتم **RBFS**

 **ساختار برنامه و گراف از پیش آماده شده‌اند. فقط کافی‌ست توابع مربوط به الگوریتم‌ها را کامل کنید.**


In [1]:
import math
# گراف مسیرها
map_graph = {
    'PixelTown': {'BitCity': 4, 'CodeValley': 3},
    'BitCity': {'PixelTown': 4, 'LoopHill': 5, 'RamVillage': 7},
    'CodeValley': {'PixelTown': 3, 'StackLake': 4},
    'LoopHill': {'BitCity': 5, 'HeapBridge': 3},
    'RamVillage': {'BitCity': 7, 'CacheCliff': 5},
    'StackLake': {'CodeValley': 4, 'CacheCliff': 6},
    'HeapBridge': {'LoopHill': 3, 'ControlBase': 8},
    'CacheCliff': {'RamVillage': 5, 'StackLake': 6, 'ControlBase': 7},
    'ControlBase': {}
}

# هیورستیک تا ControlBase
heuristic = {
    'PixelTown': 10,
    'BitCity': 9,
    'CodeValley': 9,
    'LoopHill': 6,
    'RamVillage': 6,
    'StackLake': 7,
    'HeapBridge': 3,
    'CacheCliff': 3,
    'ControlBase': 0
}


In [2]:

def ida_star(start, goal):
    """
    پیاده‌سازی الگوریتم IDA*
    ورودی: گره شروع و هدف
    خروجی: مسیر بهینه به‌صورت لیست از شهرها
    """
    def dfs(current, goal, g, threshold, path, visited):
        f = g + heuristic[current]
        if f > threshold:
            return None, f

        path.append(current)
        visited.add(current)

        if current == goal:
            return path, f

        min_threshold = math.inf

        for neighbor, cost in map_graph[current].items():
            if neighbor not in visited:
                result, temp_threshold = dfs(neighbor, goal, g + cost, threshold, path.copy(), visited.copy())
                if result is not None:
                    return result, temp_threshold
                min_threshold = min(min_threshold, temp_threshold)

        return None, min_threshold

    threshold = heuristic[start]

    while True:
        result, temp_threshold = dfs(start, goal, 0, threshold, [], set())
        if result is not None:
            return result
        if temp_threshold == math.inf:
            return None 
        threshold = temp_threshold
    pass


In [15]:

def rbfs(start, goal):
    """
    پیاده‌سازی الگوریتم RBFS
    ورودی: گره شروع و هدف
    خروجی: مسیر بهینه و هزینه کل
    """
    def recursive_rbfs(current, g, f_limit, path, visited):
        f = max(g + heuristic[current], f_limit)  

        if f > f_limit:
            return None, f, None

        path.append(current)
        visited.add(current)

        if current == goal:
            return path, f, g
        
        if not map_graph[current]:
            return None, math.inf, None
    
        children = []
        for neighbor, cost in map_graph[current].items():
            if neighbor not in visited:
                child_g = g + cost
                child_f = child_g + heuristic[neighbor]
                children.append((neighbor, child_g, child_f))

        if not children:
            return None, math.inf, None

        while True:
            children.sort(key=lambda x: x[2])
            best = children[0]

            if best[2] > f_limit:
                return None, best[2], None

            alt_f = children[1][2] if len(children) > 1 else math.inf

            result, best_f, cost = recursive_rbfs(best[0], best[1], min(f_limit, alt_f), path.copy(), visited.copy())

            children[0] = (best[0], best[1], best_f)

            if result is not None:
                return result, best_f, cost

    result, _, cost = recursive_rbfs(start, 0, math.inf, [], set())
    return result, cost
    pass


In [16]:

# دریافت ورودی و اجرای الگوریتم‌ها
start_city = input("لطفاً نام شهر مبدأ را وارد کنید: ").strip()

# اجرای IDA*
path_ida = ida_star(start_city, 'ControlBase')
cost_ida = sum(map_graph[path_ida[i]][path_ida[i+1]] for i in range(len(path_ida)-1))
print("روش IDA*:")
print("مسیر:", " -> ".join(path_ida))
print("هزینه کل:", cost_ida)

# اجرای RBFS
path_rbfs, cost_rbfs = rbfs(start_city, 'ControlBase')
print("\nروش RBFS:")
print("مسیر:", " -> ".join(path_rbfs))
print("هزینه کل:", cost_rbfs)


لطفاً نام شهر مبدأ را وارد کنید:  BitCity


روش IDA*:
مسیر: BitCity -> LoopHill -> HeapBridge -> ControlBase
هزینه کل: 16


ValueError: not enough values to unpack (expected 3, got 2)